In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 23)

df = pd.read_csv('./recipeData.csv', encoding='latin-1')

In [7]:
df = df[df['StyleID'].isin(df['StyleID'].value_counts()[df['StyleID'].value_counts() > 1000].index)]

In [8]:
df = df.drop(['BeerID', 'Name', 'URL', 'Style', 'UserId', 'PrimingMethod', 'PrimingAmount'], axis=1)

In [9]:
df['SugarScale'] = df['SugarScale'].map({'Specific Gravity': 0, 'Plato': 1})

In [10]:
df = pd.get_dummies(df, columns=['BrewMethod'])

In [11]:
df.head()

,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,SugarScale,PitchRate,PrimaryTemp,BrewMethod_All Grain,BrewMethod_BIAB,BrewMethod_Partial Mash,BrewMethod_extract
2,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,NaN,70.0,NaN,0,NaN,NaN,0,0,0,1
3,7,22.71,1.061,1.017,5.80,54.48,8.50,26.50,60,NaN,70.0,NaN,0,NaN,NaN,1,0,0,0
5,10,24.61,1.055,1.013,5.58,40.12,8.00,29.34,70,1.047,79.0,NaN,0,1.0,NaN,1,0,0,0
6,86,22.71,1.072,1.018,7.09,268.71,6.33,30.28,90,NaN,75.0,NaN,0,NaN,NaN,1,0,0,0
9,86,20.82,1.080,1.017,8.22,93.02,8.29,28.39,60,1.058,70.0,NaN,0,NaN,21.11,1,0,0,0


In [12]:
df.isna().sum()

StyleID                        0
Size(L)                        0
OG                             0
FG                             0
ABV                            0
IBU                            0
Color                          0
BoilSize                       0
BoilTime                       0
BoilGravity                 1300
Efficiency                     0
MashThickness              14495
SugarScale                     0
PitchRate                  19549
PrimaryTemp                11265
BrewMethod_All Grain           0
BrewMethod_BIAB                0
BrewMethod_Partial Mash        0
BrewMethod_extract             0
dtype: int64

In [13]:
df['BoilGravity'] = df['BoilGravity'].fillna(df['BoilGravity'].median())

In [14]:
X_train = df[df['PitchRate'].notnull()]
X_train = X_train.drop(['PitchRate'], axis=1)
y_train = df['PitchRate'][df['PitchRate'].notnull()]
X_test = df[df['PitchRate'].isnull()]
y_test = df['PitchRate'][df['PitchRate'].isnull()]
X_test = X_test.drop(['PitchRate'], axis=1)

In [15]:
X_train.drop(['MashThickness'], axis=1, inplace=True)
X_test.drop(['MashThickness'], axis=1, inplace=True)
X_train.drop(['PrimaryTemp'], axis=1, inplace=True)
X_test.drop(['PrimaryTemp'], axis=1, inplace=True)

In [16]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(random_state=28064212)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [17]:
y_pred

array([0.5 , 1.25, 1.25, ..., 1.25, 0.35, 1.25])

In [18]:
df['PitchRate'][df['PitchRate'].isnull()] = y_pred

/tmp/ipykernel_29676/1661339463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PitchRate'][df['PitchRate'].isnull()] = y_pred


In [20]:
df.isna().sum()

StyleID                        0
Size(L)                        0
OG                             0
FG                             0
ABV                            0
IBU                            0
Color                          0
BoilSize                       0
BoilTime                       0
BoilGravity                    0
Efficiency                     0
MashThickness              14495
SugarScale                     0
PitchRate                      0
PrimaryTemp                11265
BrewMethod_All Grain           0
BrewMethod_BIAB                0
BrewMethod_Partial Mash        0
BrewMethod_extract             0
dtype: int64